# 10 — SQL Analytics with DuckDB

**DuckDB** is an in-process OLAP database — like SQLite, but optimized for **analytics**.  
It runs SQL directly on CSV files, Parquet files, and Pandas DataFrames with zero setup.

### What you'll learn
| # | Topic | Key Takeaway |
|---|-------|--------------|
| 1 | **Setup** | Initialize DuckDB in-memory and persistent modes |
| 2 | **CSV Ingestion** | Load large CSVs with `read_csv_auto`, schema inference |
| 3 | **Query DataFrames** | Run SQL directly on Pandas DataFrames (zero copy!) |
| 4 | **Query Files** | Run SQL on CSV/Parquet files without loading into memory |
| 5 | **Parquet Integration** | Seamlessly move data between DuckDB and Parquet |
| 6 | **OLAP Features** | Window functions, CTEs, complex aggregations |
| 7 | **Export & Persist** | `COPY TO` CSV, save to a `.db` file |

---
## 1. Setup — Initializing DuckDB

DuckDB can run **in-memory** (fast, temporary) or **persistent** (saved to a `.db` file).  
No server needed — it runs inside your Python process.

In [1]:
import duckdb

# ============================================================
# 1a. In-Memory Connection
# ============================================================
# duckdb.connect() creates an in-memory database.
# All data lives in RAM and is lost when the connection is closed.
# Perfect for ad-hoc analytics and exploration.

con = duckdb.connect()  # or duckdb.connect(':memory:') — same thing

# Check DuckDB version
result = con.sql("SELECT version() AS duckdb_version")
# con.sql() executes a SQL query and returns a DuckDBPyRelation object

print(result)
print(f"\nDuckDB version: {duckdb.__version__}")

┌────────────────┐
│ duckdb_version │
│    varchar     │
├────────────────┤
│ v1.4.4         │
└────────────────┘


DuckDB version: 1.4.4


In [2]:
# ============================================================
# 1b. Basic SQL execution methods
# ============================================================
# DuckDB provides multiple ways to get results:

# .sql() returns a relation (lazy — not executed until you fetch)
rel = con.sql("SELECT 42 AS answer, 'hello' AS greeting")

# .fetchall() → list of tuples (raw Python)
print("fetchall()  :", rel.fetchall())

# .fetchone() → single tuple (first row only)
rel2 = con.sql("SELECT 42 AS answer")
print("fetchone()  :", rel2.fetchone())

# .fetchdf() → Pandas DataFrame (most useful for analysis)
df_result = con.sql("SELECT 42 AS answer, 'hello' AS greeting").fetchdf()
# fetchdf() converts the result directly to a Pandas DataFrame
print("fetchdf()   :")
print(df_result)

# .show() → print formatted output to console (quick peek)
print("\nshow():")
con.sql("SELECT 42 AS answer, 'hello' AS greeting").show()

fetchall()  : [(42, 'hello')]
fetchone()  : (42,)
fetchdf()   :
   answer greeting
0      42    hello

show():
┌────────┬──────────┐
│ answer │ greeting │
│ int32  │ varchar  │
├────────┼──────────┤
│     42 │ hello    │
└────────┴──────────┘



---
## 2. CSV Ingestion — Loading Large CSVs

DuckDB's `read_csv_auto` is extremely fast and auto-detects:  
column names, data types, delimiters, date formats, and more.

In [5]:
# ============================================================
# 2a. read_csv_auto — auto-detect everything
# ============================================================
# read_csv_auto(path) reads a CSV with automatic schema inference.
# DuckDB samples the file to guess types — much smarter than Pandas.

# Quick peek at the CSV structure (first 5 rows)
con.sql("""
    SELECT *
    FROM read_csv_auto('data/hotel_booking.csv')
    LIMIT 5
""").show()

┌──────────────┬─────────────┬───────────┬───────────────────┬────────────────────┬──────────────────────────┬───────────────────────────┬─────────────────────────┬──────────────────────┬────────┬──────────┬────────┬─────────┬─────────┬────────────────┬──────────────────────┬───────────────────┬────────────────────────┬────────────────────────────────┬────────────────────┬────────────────────┬─────────────────┬──────────────┬────────┬─────────┬──────────────────────┬───────────────┬────────┬─────────────────────────────┬───────────────────────────┬────────────────────┬─────────────────────────┬────────────────┬─────────────────────────────┬──────────────┬──────────────────┐
│    hotel     │ is_canceled │ lead_time │ arrival_date_year │ arrival_date_month │ arrival_date_week_number │ arrival_date_day_of_month │ stays_in_weekend_nights │ stays_in_week_nights │ adults │ children │ babies │  meal   │ country │ market_segment │ distribution_channel │ is_repeated_guest │ previous_cancellatio

In [6]:
# ============================================================
# 2b. Inspect auto-detected schema
# ============================================================
# DESCRIBE shows column names and inferred data types.
# Useful for verifying DuckDB guessed correctly.

con.sql("""
    DESCRIBE SELECT *
    FROM read_csv_auto('data/hotel_booking.csv')
""").show()

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ hotel                       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ is_canceled                 │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ lead_time                   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ arrival_date_year           │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ arrival_date_month          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ arrival_date_week_number    │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ arrival_date_day_of_month   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ stays_in_weekend_nights     │ BIGINT      │ YES     

In [7]:
# ============================================================
# 2c. CREATE TABLE from CSV — persist in DuckDB
# ============================================================
# CREATE TABLE AS SELECT (CTAS) loads CSV data into a DuckDB table.
# Once loaded, queries run on DuckDB's optimized columnar format.

con.sql("""
    CREATE OR REPLACE TABLE bookings AS
    SELECT *
    FROM read_csv_auto('data/hotel_booking.csv')
""")
# CREATE OR REPLACE → drop the table if it exists and recreate

# Verify row count
con.sql("SELECT COUNT(*) AS total_rows FROM bookings").show()

┌────────────┐
│ total_rows │
│   int64    │
├────────────┤
│     119390 │
└────────────┘



In [8]:
# ============================================================
# 2d. read_csv with explicit options
# ============================================================
# For full control, you can specify options manually:
#   delim       : column separator (default: auto-detect)
#   header      : True/False — does the file have a header row?
#   columns     : dict of {name: type} for explicit schema
#   nullstr     : string to interpret as NULL
#   sample_size : rows to sample for type inference (default: -1 = all)

# Example: read with explicit settings
con.sql("""
    SELECT hotel, country, adr, lead_time
    FROM read_csv(
        'data/hotel_booking.csv',
        delim = ',',
        header = true,
        nullstr = '',
        auto_detect = true
    )
    LIMIT 5
""").show()

┌──────────────┬─────────┬────────┬───────────┐
│    hotel     │ country │  adr   │ lead_time │
│   varchar    │ varchar │ double │   int64   │
├──────────────┼─────────┼────────┼───────────┤
│ Resort Hotel │ PRT     │    0.0 │       342 │
│ Resort Hotel │ PRT     │    0.0 │       737 │
│ Resort Hotel │ GBR     │   75.0 │         7 │
│ Resort Hotel │ GBR     │   75.0 │        13 │
│ Resort Hotel │ GBR     │   98.0 │        14 │
└──────────────┴─────────┴────────┴───────────┘



---
## 3. Query Pandas DataFrames — Zero Copy SQL

DuckDB can query Pandas DataFrames **directly** by name — no loading or copying needed.  
Just reference the Python variable name in your SQL `FROM` clause.

In [9]:
import pandas as pd

# ============================================================
# 3a. Query a DataFrame by variable name
# ============================================================
# Load a DataFrame with Pandas (as we did in notebook 04)
df_hotel = pd.read_csv(
    "data/hotel_booking.csv",
    usecols=["hotel", "is_canceled", "lead_time", "country", "adr",
             "arrival_date_year", "arrival_date_month",
             "stays_in_weekend_nights", "stays_in_week_nights",
             "reserved_room_type", "market_segment"],
)

print(f"DataFrame shape: {df_hotel.shape}")

# Now run SQL on the DataFrame — just use the variable name!
# DuckDB automatically detects Python variables as virtual tables.
con.sql("""
    SELECT hotel, COUNT(*) AS bookings, ROUND(AVG(adr), 2) AS avg_adr
    FROM df_hotel
    GROUP BY hotel
""").show()

DataFrame shape: (119390, 11)
┌──────────────┬──────────┬─────────┐
│    hotel     │ bookings │ avg_adr │
│   varchar    │  int64   │ double  │
├──────────────┼──────────┼─────────┤
│ City Hotel   │    79330 │   105.3 │
│ Resort Hotel │    40060 │   94.95 │
└──────────────┴──────────┴─────────┘



In [10]:
# ============================================================
# 3b. Complex query on DataFrame — filter, aggregate, sort
# ============================================================

result = con.sql("""
    SELECT
        country,
        COUNT(*) AS total_bookings,
        SUM(CASE WHEN is_canceled = 1 THEN 1 ELSE 0 END) AS cancellations,
        ROUND(AVG(adr), 2) AS avg_adr,
        ROUND(AVG(lead_time), 0) AS avg_lead_time
    FROM df_hotel
    WHERE country IS NOT NULL
    GROUP BY country
    HAVING COUNT(*) > 1000
    ORDER BY total_bookings DESC
    LIMIT 10
""").fetchdf()  # fetchdf() converts result → Pandas DataFrame

print(result.to_string(index=False))

country  total_bookings  cancellations  avg_adr  avg_lead_time
    PRT           48590        27519.0    92.04          116.0
    GBR           12129         2453.0    96.02          127.0
    FRA           10415         1934.0   109.62           82.0
    ESP            8568         2177.0   117.00           55.0
    DEU            7287         1218.0   104.40          137.0
    ITA            3766         1333.0   113.95           91.0
    IRL            3375          832.0    98.19          120.0
    BEL            2342          474.0   113.85          100.0
    BRA            2224          830.0   111.01           83.0
    NLD            2104          387.0   108.09           81.0


In [11]:
# ============================================================
# 3c. Join DataFrames with SQL
# ============================================================
# You can join multiple DataFrames together using SQL.

# Create a dimension DataFrame
df_countries = pd.DataFrame({
    "code": ["PRT", "GBR", "FRA", "ESP", "DEU", "ITA", "BRA", "USA", "NLD", "IRL"],
    "country_name": ["Portugal", "United Kingdom", "France", "Spain",
                     "Germany", "Italy", "Brazil", "United States",
                     "Netherlands", "Ireland"],
    "continent": ["Europe", "Europe", "Europe", "Europe",
                  "Europe", "Europe", "South America", "North America",
                  "Europe", "Europe"],
})

# SQL JOIN across two DataFrames — just use their variable names
con.sql("""
    SELECT
        c.country_name,
        c.continent,
        COUNT(*) AS bookings,
        ROUND(AVG(h.adr), 2) AS avg_adr
    FROM df_hotel h
    JOIN df_countries c ON h.country = c.code
    WHERE h.is_canceled = 0
    GROUP BY c.country_name, c.continent
    ORDER BY bookings DESC
""").show()

┌────────────────┬───────────────┬──────────┬─────────┐
│  country_name  │   continent   │ bookings │ avg_adr │
│    varchar     │    varchar    │  int64   │ double  │
├────────────────┼───────────────┼──────────┼─────────┤
│ Portugal       │ Europe        │    21071 │   90.34 │
│ United Kingdom │ Europe        │     9676 │   90.47 │
│ France         │ Europe        │     8481 │  105.75 │
│ Spain          │ Europe        │     6391 │   110.3 │
│ Germany        │ Europe        │     6069 │  101.26 │
│ Ireland        │ Europe        │     2543 │   94.69 │
│ Italy          │ Europe        │     2433 │  110.76 │
│ Netherlands    │ Europe        │     1717 │  105.01 │
│ United States  │ North America │     1596 │  118.98 │
│ Brazil         │ South America │     1394 │   107.6 │
├────────────────┴───────────────┴──────────┴─────────┤
│ 10 rows                                   4 columns │
└─────────────────────────────────────────────────────┘



---
## 4. Query Files Directly — No Loading Required

DuckDB can run SQL directly on **CSV and Parquet files** on disk.  
The file is never fully loaded into memory — DuckDB streams and processes it.

In [12]:
# ============================================================
# 4a. Query CSV file directly
# ============================================================
# Just put the file path in read_csv_auto() inside the FROM clause.
# No CREATE TABLE, no pd.read_csv — straight to results.

con.sql("""
    SELECT
        hotel,
        arrival_date_year AS year,
        COUNT(*) AS bookings,
        ROUND(SUM(adr * (stays_in_weekend_nights + stays_in_week_nights)), 2) AS total_revenue
    FROM read_csv_auto('data/hotel_booking.csv')
    WHERE is_canceled = 0
    GROUP BY hotel, arrival_date_year
    ORDER BY hotel, year
""").show()

┌──────────────┬───────┬──────────┬───────────────┐
│    hotel     │ year  │ bookings │ total_revenue │
│   varchar    │ int64 │  int64   │    double     │
├──────────────┼───────┼──────────┼───────────────┤
│ City Hotel   │  2015 │     7678 │    1894323.54 │
│ City Hotel   │  2016 │    22733 │     6862128.0 │
│ City Hotel   │  2017 │    15817 │    5637958.64 │
│ Resort Hotel │  2015 │     6176 │    2617235.57 │
│ Resort Hotel │  2016 │    13637 │    4811373.43 │
│ Resort Hotel │  2017 │     9125 │    4173241.23 │
└──────────────┴───────┴──────────┴───────────────┘



In [13]:
# ============================================================
# 4b. Query Parquet file directly
# ============================================================
# First, create a Parquet file from the CSV (if not already done in notebook 04)

from pathlib import Path

output_dir = Path("data/output")
output_dir.mkdir(parents=True, exist_ok=True)
parquet_path = output_dir / "hotel_bookings.parquet"

# Use DuckDB itself to convert CSV → Parquet (fast!)
# COPY ... TO ... writes query results to a file.
con.sql(f"""
    COPY (
        SELECT * FROM read_csv_auto('data/hotel_booking.csv')
    )
    TO '{parquet_path}'
    (FORMAT PARQUET, COMPRESSION SNAPPY)
""")
# FORMAT PARQUET  → output format
# COMPRESSION SNAPPY → fast compression algorithm

csv_size = Path("data/hotel_booking.csv").stat().st_size / 1024**2
pq_size = parquet_path.stat().st_size / 1024**2
print(f"CSV size     : {csv_size:.2f} MB")
print(f"Parquet size : {pq_size:.2f} MB")
print(f"Compression  : {csv_size / pq_size:.1f}x smaller")

CSV size     : 23.95 MB
Parquet size : 5.11 MB
Compression  : 4.7x smaller


In [14]:
# Now query the Parquet file directly — even faster than CSV
# read_parquet() or just the file path with a .parquet extension

con.sql(f"""
    SELECT
        hotel,
        reserved_room_type AS room,
        COUNT(*) AS bookings,
        ROUND(AVG(adr), 2) AS avg_adr,
        ROUND(AVG(lead_time), 0) AS avg_lead_days
    FROM read_parquet('{parquet_path}')
    WHERE is_canceled = 0
      AND adr > 0
    GROUP BY hotel, reserved_room_type
    ORDER BY hotel, bookings DESC
""").show()

┌──────────────┬─────────┬──────────┬─────────┬───────────────┐
│    hotel     │  room   │ bookings │ avg_adr │ avg_lead_days │
│   varchar    │ varchar │  int64   │ double  │    double     │
├──────────────┼─────────┼──────────┼─────────┼───────────────┤
│ City Hotel   │ A       │    34529 │   98.84 │          85.0 │
│ City Hotel   │ D       │     7513 │  129.93 │          72.0 │
│ City Hotel   │ F       │     1060 │  190.53 │          67.0 │
│ City Hotel   │ E       │     1011 │  159.03 │          59.0 │
│ City Hotel   │ B       │      717 │   91.01 │         109.0 │
│ City Hotel   │ G       │      313 │  226.58 │          55.0 │
│ City Hotel   │ C       │        6 │  102.17 │          98.0 │
│ Resort Hotel │ A       │    16567 │   75.81 │          76.0 │
│ Resort Hotel │ D       │     5406 │   101.4 │          92.0 │
│ Resort Hotel │ E       │     3511 │  110.92 │          91.0 │
│ Resort Hotel │ G       │      937 │  164.71 │          68.0 │
│ Resort Hotel │ F       │      891 │  1

In [15]:
# ============================================================
# 4c. Glob patterns — query multiple files at once
# ============================================================
# DuckDB supports glob patterns to query many files as one table.
# Useful for: partitioned datasets, daily batch files.
#
# read_csv_auto('data/output/*.csv')   → all CSVs in the folder
# read_parquet('data/year=*/data.parquet') → Hive-partitioned parquet

# Query ALL Parquet files in output/ as if they were one table
con.sql("""
    SELECT COUNT(*) AS total_rows
    FROM read_parquet('data/output/*.parquet')
""").show()

┌────────────┐
│ total_rows │
│   int64    │
├────────────┤
│     119390 │
└────────────┘



---
## 5. Parquet Integration — DuckDB as a Parquet Engine

DuckDB reads and writes Parquet natively. It can also inspect Parquet metadata  
and perform **column pruning** (read only needed columns from Parquet files).

In [16]:
# ============================================================
# 5a. Inspect Parquet metadata from SQL
# ============================================================
# parquet_metadata() reads the Parquet file footer — no data loaded.
# Shows: row groups, row count, total size, etc.

con.sql(f"""
    SELECT
        row_group_id,
        row_group_num_rows,
        row_group_bytes
    FROM parquet_metadata('{parquet_path}')
""").show()

┌──────────────┬────────────────────┬─────────────────┐
│ row_group_id │ row_group_num_rows │ row_group_bytes │
│    int64     │       int64        │      int64      │
├──────────────┼────────────────────┼─────────────────┤
│            0 │             119390 │         9350169 │
│            0 │             119390 │         9350169 │
│            0 │             119390 │         9350169 │
│            0 │             119390 │         9350169 │
│            0 │             119390 │         9350169 │
│            0 │             119390 │         9350169 │
│            0 │             119390 │         9350169 │
│            0 │             119390 │         9350169 │
│            0 │             119390 │         9350169 │
│            0 │             119390 │         9350169 │
│            · │                ·   │            ·    │
│            · │                ·   │            ·    │
│            · │                ·   │            ·    │
│            0 │             119390 │         93

In [17]:
# ============================================================
# 5b. Inspect Parquet schema
# ============================================================
# parquet_schema() shows column names, types, and compression.

con.sql(f"""
    SELECT name, type, converted_type
    FROM parquet_schema('{parquet_path}')
    LIMIT 15
""").show()

┌───────────────────────────┬────────────┬────────────────┐
│           name            │    type    │ converted_type │
│          varchar          │  varchar   │    varchar     │
├───────────────────────────┼────────────┼────────────────┤
│ duckdb_schema             │ NULL       │ NULL           │
│ hotel                     │ BYTE_ARRAY │ UTF8           │
│ is_canceled               │ INT64      │ INT_64         │
│ lead_time                 │ INT64      │ INT_64         │
│ arrival_date_year         │ INT64      │ INT_64         │
│ arrival_date_month        │ BYTE_ARRAY │ UTF8           │
│ arrival_date_week_number  │ INT64      │ INT_64         │
│ arrival_date_day_of_month │ INT64      │ INT_64         │
│ stays_in_weekend_nights   │ INT64      │ INT_64         │
│ stays_in_week_nights      │ INT64      │ INT_64         │
│ adults                    │ INT64      │ INT_64         │
│ children                  │ DOUBLE     │ NULL           │
│ babies                    │ INT64     

In [18]:
# ============================================================
# 5c. Parquet → DuckDB Table → Parquet round-trip
# ============================================================
# Load Parquet into a table, transform, export back to Parquet.
# A common ETL pattern: raw.parquet → transform → clean.parquet

# Step 1: Load from Parquet into a table
con.sql(f"""
    CREATE OR REPLACE TABLE hotel_clean AS
    SELECT
        hotel,
        country,
        CAST(adr AS FLOAT) AS adr,
        -- CAST(value AS type) converts a value to a specific SQL data type
        (stays_in_weekend_nights + stays_in_week_nights) AS total_nights,
        adr * (stays_in_weekend_nights + stays_in_week_nights) AS total_revenue,
        arrival_date_year AS year,
        arrival_date_month AS month,
        reserved_room_type AS room_type,
        market_segment,
        is_canceled
    FROM read_parquet('{parquet_path}')
    WHERE adr > 0
      AND adr < 5000   -- remove outliers
""")

con.sql("SELECT COUNT(*) AS rows FROM hotel_clean").show()

# Step 2: Export the cleaned table back to Parquet
clean_parquet = output_dir / "hotel_clean.parquet"
con.sql(f"""
    COPY hotel_clean
    TO '{clean_parquet}'
    (FORMAT PARQUET, COMPRESSION ZSTD)
""")
# COMPRESSION ZSTD → Zstandard: better ratio than Snappy, nearly as fast

print(f"Clean Parquet: {clean_parquet.stat().st_size / 1024**2:.2f} MB")

┌────────┐
│  rows  │
│ int64  │
├────────┤
│ 117429 │
└────────┘

Clean Parquet: 0.59 MB


---
## 6. OLAP Features — Window Functions & Advanced SQL

Window functions perform calculations **across rows** related to the current row,  
without collapsing them into a single output row (unlike GROUP BY).

In [19]:
# ============================================================
# 6a. ROW_NUMBER — assign sequential IDs within partitions
# ============================================================
# ROW_NUMBER() OVER (PARTITION BY ... ORDER BY ...)
#   PARTITION BY → groups rows (like GROUP BY but keeps all rows)
#   ORDER BY     → determines numbering order within each partition
#
# Use case: find the top-N items per group.

con.sql("""
    WITH ranked AS (
        -- CTE (Common Table Expression): a named temporary result set
        -- WITH name AS (query) makes complex queries readable
        SELECT
            hotel,
            country,
            COUNT(*) AS bookings,
            ROUND(AVG(adr), 2) AS avg_adr,
            ROW_NUMBER() OVER (
                PARTITION BY hotel     -- restart numbering for each hotel
                ORDER BY COUNT(*) DESC -- rank by most bookings first
            ) AS rank
        FROM hotel_clean
        WHERE is_canceled = 0
        GROUP BY hotel, country
    )
    SELECT *
    FROM ranked
    WHERE rank <= 5   -- top 5 countries per hotel
    ORDER BY hotel, rank
""").show()

┌──────────────┬─────────┬──────────┬─────────┬───────┐
│    hotel     │ country │ bookings │ avg_adr │ rank  │
│   varchar    │ varchar │  int64   │ double  │ int64 │
├──────────────┼─────────┼──────────┼─────────┼───────┤
│ City Hotel   │ PRT     │     9993 │   98.17 │     1 │
│ City Hotel   │ FRA     │     7042 │   109.1 │     2 │
│ City Hotel   │ DEU     │     4982 │  107.04 │     3 │
│ City Hotel   │ GBR     │     3737 │  112.64 │     4 │
│ City Hotel   │ ESP     │     3255 │  110.83 │     5 │
│ Resort Hotel │ PRT     │     9715 │   94.97 │     1 │
│ Resort Hotel │ GBR     │     5867 │   77.46 │     2 │
│ Resort Hotel │ ESP     │     3058 │  112.55 │     3 │
│ Resort Hotel │ IRL     │     1729 │   86.76 │     4 │
│ Resort Hotel │ FRA     │     1386 │    92.8 │     5 │
├──────────────┴─────────┴──────────┴─────────┴───────┤
│ 10 rows                                   5 columns │
└─────────────────────────────────────────────────────┘



In [20]:
# ============================================================
# 6b. Running Totals — SUM OVER with window frame
# ============================================================
# SUM() OVER (ORDER BY ...) computes a cumulative/running total.
#   ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW → running total
#   This is the DEFAULT frame for ORDER BY, so it's often omitted.

con.sql("""
    SELECT
        year,
        month,
        SUM(total_revenue) AS monthly_revenue,
        SUM(SUM(total_revenue)) OVER (
            PARTITION BY year
            ORDER BY MIN(total_revenue)  -- rough month ordering
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS running_total
    FROM hotel_clean
    WHERE is_canceled = 0
      AND year = 2016
    GROUP BY year, month
    ORDER BY monthly_revenue
""").show()

┌───────┬───────────┬────────────────────┬────────────────────┐
│ year  │   month   │  monthly_revenue   │   running_total    │
│ int64 │  varchar  │       double       │       double       │
├───────┼───────────┼────────────────────┼────────────────────┤
│  2016 │ January   │  264521.3800000002 │  748692.1000000006 │
│  2016 │ February  │  484170.7200000003 │  484170.7200000003 │
│  2016 │ December  │  657870.7199999993 │  9455423.420000004 │
│  2016 │ November  │  688843.4699999993 │  7725450.860000007 │
│  2016 │ March     │  767337.4199999995 │         1516029.52 │
│  2016 │ April     │   896591.380000001 │  7036607.390000008 │
│  2016 │ October   │ 1072101.8399999987 │  8797552.700000005 │
│  2016 │ May       │  1073277.629999999 │ 11673501.430000002 │
│  2016 │ June      │ 1144800.3799999994 │ 10600223.800000003 │
│  2016 │ September │ 1289642.6900000002 │  4614996.960000005 │
│  2016 │ July      │  1525019.050000003 │  6140016.010000007 │
│  2016 │ August    │ 1809324.7500000042

In [21]:
# ============================================================
# 6c. LAG / LEAD — access previous / next row values
# ============================================================
# LAG(column, offset, default) → value from N rows BEFORE
# LEAD(column, offset, default) → value from N rows AFTER
#
# Use case: compare current value vs. previous period (MoM, YoY).

con.sql("""
    WITH yearly AS (
        SELECT
            year,
            ROUND(SUM(total_revenue), 2) AS revenue,
            COUNT(*) AS bookings
        FROM hotel_clean
        WHERE is_canceled = 0
        GROUP BY year
    )
    SELECT
        year,
        revenue,
        bookings,
        LAG(revenue, 1) OVER (ORDER BY year) AS prev_year_revenue,
        -- LAG(revenue, 1) looks 1 row back ordered by year
        ROUND(
            (revenue - LAG(revenue, 1) OVER (ORDER BY year))
            / LAG(revenue, 1) OVER (ORDER BY year) * 100,
            1
        ) AS yoy_growth_pct
        -- Year-over-Year growth percentage
    FROM yearly
    ORDER BY year
""").show()

┌───────┬─────────────┬──────────┬───────────────────┬────────────────┐
│ year  │   revenue   │ bookings │ prev_year_revenue │ yoy_growth_pct │
│ int64 │   double    │  int64   │      double       │     double     │
├───────┼─────────────┼──────────┼───────────────────┼────────────────┤
│  2015 │  4511559.11 │    13337 │              NULL │           NULL │
│  2016 │ 11673501.43 │    35544 │        4511559.11 │          158.7 │
│  2017 │  9811263.67 │    24538 │       11673501.43 │          -16.0 │
└───────┴─────────────┴──────────┴───────────────────┴────────────────┘



In [22]:
# ============================================================
# 6d. RANK and NTILE — ranking and bucketing
# ============================================================
# RANK()    → rank with gaps (1, 2, 2, 4)
# DENSE_RANK() → rank without gaps (1, 2, 2, 3)
# NTILE(n)  → split rows into N equal-sized buckets (quartiles, deciles...)

con.sql("""
    SELECT
        country,
        total_bookings,
        avg_adr,
        RANK() OVER (ORDER BY total_bookings DESC) AS rank,
        NTILE(4) OVER (ORDER BY avg_adr) AS adr_quartile
        -- NTILE(4) splits all rows into 4 equal buckets by avg_adr
        -- Quartile 1 = lowest ADR, Quartile 4 = highest ADR
    FROM (
        SELECT
            country,
            COUNT(*) AS total_bookings,
            ROUND(AVG(adr), 2) AS avg_adr
        FROM hotel_clean
        WHERE is_canceled = 0
        GROUP BY country
        HAVING COUNT(*) > 500
    )
    ORDER BY rank
    LIMIT 10
""").show()

┌─────────┬────────────────┬─────────┬───────┬──────────────┐
│ country │ total_bookings │ avg_adr │ rank  │ adr_quartile │
│ varchar │     int64      │ double  │ int64 │    int64     │
├─────────┼────────────────┼─────────┼───────┼──────────────┤
│ PRT     │          19708 │   96.59 │     1 │            1 │
│ GBR     │           9604 │   91.15 │     2 │            1 │
│ FRA     │           8428 │  106.42 │     3 │            3 │
│ ESP     │           6313 │  111.67 │     4 │            4 │
│ DEU     │           6028 │  101.95 │     5 │            1 │
│ IRL     │           2537 │   94.91 │     6 │            1 │
│ ITA     │           2417 │  111.49 │     7 │            3 │
│ BEL     │           1862 │  111.84 │     8 │            4 │
│ NLD     │           1713 │  105.26 │     9 │            2 │
│ USA     │           1586 │  119.73 │    10 │            4 │
├─────────┴────────────────┴─────────┴───────┴──────────────┤
│ 10 rows                                         5 columns │
└───────

In [23]:
# ============================================================
# 6e. Complex CTE pipeline — multi-step analytics
# ============================================================
# CTEs let you break a complex query into readable, named steps.
# Each CTE can reference the previous one.

con.sql("""
    -- Step 1: Base metrics per hotel + room type
    WITH room_stats AS (
        SELECT
            hotel,
            room_type,
            COUNT(*) AS bookings,
            ROUND(AVG(adr), 2) AS avg_adr,
            ROUND(SUM(total_revenue), 2) AS total_revenue,
            ROUND(AVG(total_nights), 1) AS avg_stay_nights
        FROM hotel_clean
        WHERE is_canceled = 0
        GROUP BY hotel, room_type
    ),

    -- Step 2: Add hotel-level totals for percentage calculation
    hotel_totals AS (
        SELECT
            hotel,
            SUM(bookings) AS hotel_total_bookings
        FROM room_stats
        GROUP BY hotel
    ),

    -- Step 3: Combine and compute percentages
    enriched AS (
        SELECT
            r.*,
            ROUND(r.bookings * 100.0 / h.hotel_total_bookings, 1) AS pct_of_hotel
        FROM room_stats r
        JOIN hotel_totals h ON r.hotel = h.hotel
    )

    -- Final: show top room types by revenue per hotel
    SELECT *
    FROM enriched
    WHERE bookings > 100
    ORDER BY hotel, total_revenue DESC
""").show()

┌──────────────┬───────────┬──────────┬─────────┬───────────────┬─────────────────┬──────────────┐
│    hotel     │ room_type │ bookings │ avg_adr │ total_revenue │ avg_stay_nights │ pct_of_hotel │
│   varchar    │  varchar  │  int64   │ double  │    double     │     double      │    double    │
├──────────────┼───────────┼──────────┼─────────┼───────────────┼─────────────────┼──────────────┤
│ City Hotel   │ A         │    34529 │   98.84 │     9632688.5 │             2.8 │         76.5 │
│ City Hotel   │ D         │     7513 │  129.93 │    3221588.26 │             3.4 │         16.6 │
│ City Hotel   │ F         │     1060 │  190.53 │     580745.11 │             2.9 │          2.3 │
│ City Hotel   │ E         │     1011 │  159.03 │     509744.91 │             3.2 │          2.2 │
│ City Hotel   │ G         │      313 │  226.58 │      234582.3 │             3.3 │          0.7 │
│ City Hotel   │ B         │      717 │   91.01 │      212992.1 │             3.3 │          1.6 │
│ Resort H

---
## 7. Export & Persist — Saving Results

DuckDB can export query results to **CSV**, **Parquet**, or **JSON**,  
and persist the full database to a `.db` file for later use.

In [ ]:
from pathlib import Path

output_dir = Path("data/output")

# ============================================================
# 7a. COPY TO — export query results to CSV
# ============================================================
# COPY (query) TO 'path' (FORMAT, HEADER, DELIMITER)
#   FORMAT CSV    → output as CSV
#   HEADER true   → include column names as first row
#   DELIMITER ',' → column separator

csv_export = output_dir / "hotel_summary_export.csv"

con.sql(f"""
    COPY (
        SELECT
            hotel,
            country,
            COUNT(*) AS bookings,
            ROUND(AVG(adr), 2) AS avg_adr,
            ROUND(SUM(total_revenue), 2) AS total_revenue
        FROM hotel_clean
        WHERE is_canceled = 0
        GROUP BY hotel, country
        HAVING COUNT(*) > 100
        ORDER BY total_revenue DESC
    )
    TO '{csv_export}'
    (FORMAT CSV, HEADER true)
""")

print(f"Exported CSV: {csv_export} ({csv_export.stat().st_size / 1024:.1f} KB)")

# Quick peek at the exported file
print("\n--- First 5 lines ---")
with csv_export.open() as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        print(line.strip())

In [ ]:
# ============================================================
# 7b. COPY TO — export to Parquet
# ============================================================

parquet_export = output_dir / "hotel_analytics.parquet"

con.sql(f"""
    COPY (
        SELECT
            hotel,
            year,
            month,
            room_type,
            market_segment,
            COUNT(*) AS bookings,
            ROUND(AVG(adr), 2) AS avg_adr,
            ROUND(SUM(total_revenue), 2) AS total_revenue,
            ROUND(AVG(total_nights), 2) AS avg_stay
        FROM hotel_clean
        WHERE is_canceled = 0
        GROUP BY hotel, year, month, room_type, market_segment
    )
    TO '{parquet_export}'
    (FORMAT PARQUET, COMPRESSION ZSTD)
""")

print(f"Exported Parquet: {parquet_export} ({parquet_export.stat().st_size / 1024:.1f} KB)")

In [25]:
# ============================================================
# 7c. Persistent Database — save to a .db file
# ============================================================
# duckdb.connect('file.db') creates a persistent database.
# Tables survive after the connection is closed.
# You can reopen it later and all data is still there.

db_path = output_dir / "hotel_analytics.db"

# Open a persistent connection
persistent_con = duckdb.connect(str(db_path))

# Create tables in the persistent database
persistent_con.sql("""
    CREATE OR REPLACE TABLE bookings AS
    SELECT * FROM read_csv_auto('data/hotel_booking.csv')
""")

persistent_con.sql("""
    CREATE OR REPLACE TABLE bookings_clean AS
    SELECT
        hotel,
        country,
        CAST(adr AS FLOAT) AS adr,
        (stays_in_weekend_nights + stays_in_week_nights) AS total_nights,
        adr * (stays_in_weekend_nights + stays_in_week_nights) AS total_revenue,
        arrival_date_year AS year,
        arrival_date_month AS month,
        reserved_room_type AS room_type,
        market_segment,
        is_canceled
    FROM bookings
    WHERE adr > 0 AND adr < 5000
""")

# Verify
persistent_con.sql("SHOW TABLES").show()
# SHOW TABLES lists all tables in the database

persistent_con.sql("""
    SELECT
        'bookings' AS table_name, COUNT(*) AS rows FROM bookings
    UNION ALL
    SELECT
        'bookings_clean', COUNT(*) FROM bookings_clean
""").show()

# Close the connection (data is saved to disk)
persistent_con.close()

print(f"\nDatabase saved: {db_path} ({db_path.stat().st_size / 1024**2:.1f} MB)")

┌────────────────┐
│      name      │
│    varchar     │
├────────────────┤
│ bookings       │
│ bookings_clean │
└────────────────┘

┌────────────────┬────────┐
│   table_name   │  rows  │
│    varchar     │ int64  │
├────────────────┼────────┤
│ bookings       │ 119390 │
│ bookings_clean │ 117429 │
└────────────────┴────────┘


Database saved: data/output/hotel_analytics.db (7.3 MB)


In [26]:
# ============================================================
# 7d. Reopen persistent database — data is still there!
# ============================================================

# Reopen the database file
reopen_con = duckdb.connect(str(db_path))

# Query the previously saved tables
reopen_con.sql("""
    SELECT hotel, COUNT(*) AS bookings, ROUND(AVG(adr), 2) AS avg_adr
    FROM bookings_clean
    WHERE is_canceled = 0
    GROUP BY hotel
""").show()

reopen_con.close()
print("Database reopened, queried, and closed successfully.")

┌──────────────┬──────────┬─────────┐
│    hotel     │ bookings │ avg_adr │
│   varchar    │  int64   │ double  │
├──────────────┼──────────┼─────────┤
│ Resort Hotel │    28270 │   92.93 │
│ City Hotel   │    45149 │  108.27 │
└──────────────┴──────────┴─────────┘

Database reopened, queried, and closed successfully.


In [ ]:
# Clean up: close the in-memory connection
con.close()
print("All connections closed.")

---
## Key Takeaways

| Feature | How | Why it matters |
|---------|-----|---------------|
| **In-memory DB** | `duckdb.connect()` | Zero-config analytics, no server |
| **CSV ingestion** | `read_csv_auto()` | Auto-detect schema, fast parallel reads |
| **Query DataFrames** | `FROM df_name` in SQL | SQL on Pandas with zero copy |
| **Query files** | `read_csv_auto('file.csv')`, `read_parquet('file.parquet')` | Analyze files without loading into memory |
| **Parquet I/O** | `COPY TO ... (FORMAT PARQUET)` | Convert, compress, and export efficiently |
| **Glob patterns** | `read_parquet('data/*.parquet')` | Query partitioned datasets |
| **Window functions** | `ROW_NUMBER`, `LAG`, `RANK`, `NTILE`, `SUM OVER` | Rankings, running totals, period comparisons |
| **CTEs** | `WITH name AS (...)` | Break complex queries into readable steps |
| **Export CSV** | `COPY (...) TO 'file.csv' (FORMAT CSV)` | Share results with non-technical users |
| **Persistent DB** | `duckdb.connect('file.db')` | Save tables to disk, reopen later |

---

## Congratulations!

You've completed the full Python for Data Engineering learning path:  
1. **File Handling** — Pathlib, CSV, JSON, JSONL, generators, binary I/O  
2. **Standard Libraries** — datetime, os/sys, logging, collections, glob  
3. **Numpy, Pandas & Parquet** — Vectorized processing, transformations, columnar storage  
4. **DuckDB SQL Analytics** — In-process OLAP, file querying, window functions, persistence